<a href="https://colab.research.google.com/github/hyeonsang010716/Recommendation-system/blob/main/latent_factor_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!wget https://files.grouplens.org/datasets/movielens/ml-latest-small.zip
!unzip ml-latest-small.zip

--2024-03-28 06:21:17--  https://files.grouplens.org/datasets/movielens/ml-latest-small.zip
Resolving files.grouplens.org (files.grouplens.org)... 128.101.65.152
Connecting to files.grouplens.org (files.grouplens.org)|128.101.65.152|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 978202 (955K) [application/zip]
Saving to: ‘ml-latest-small.zip’

ml-latest-small.zip 100%[===================>] 955.28K  4.58MB/s    in 0.2s    

2024-03-28 06:21:18 (4.58 MB/s) - ‘ml-latest-small.zip’ saved [978202/978202]

Archive:  ml-latest-small.zip
   creating: ml-latest-small/
  inflating: ml-latest-small/links.csv  
  inflating: ml-latest-small/tags.csv  
  inflating: ml-latest-small/ratings.csv  
  inflating: ml-latest-small/README.txt  
  inflating: ml-latest-small/movies.csv  


In [ ]:
import numpy as np

users = []
items = []
ratings = []

with open("ml-latest-small/ratings.csv",'r') as f:
  print(f.readline())
  for line in f:
    ul , ml , rating , _ = line.split(',')
    users.append(int(ul))
    items.append(int(ml))
    ratings.append(float(rating))

users = np.array(users)
items = np.array(items)
ratings = np.array(ratings)

userId,movieId,rating,timestamp



In [ ]:
rank = 10
alpha = ratings.mean()
user_bias = np.zeros(users.max() + 1)
item_bias = np.zeros(items.max() + 1)
user_reps = np.random.randn(users.max() + 1 , rank)
item_reps = np.random.randn(items.max() + 1 , rank)


In [ ]:
lr = 0.1
lmd = 0.001
for epoch in range(10):
  h = alpha + user_bias[users] + item_bias[items] + (user_reps[users] * item_reps[items]).sum(axis = 1)
  rmse = ((h - ratings) **2).mean() ** 0.5
  print("rmse : {} , epoch : {} ".format(rmse , epoch))
  for user , item , rating in zip(users , items , ratings):
    bu = user_bias[user]
    bi = item_bias[item]
    pu = user_reps[user]
    qi = item_reps[item]
    est = alpha + bu + bi + np.dot(pu , qi)
    diff = est - rating

    grd_user_bias = diff + lmd * bu
    grd_item_bias = diff + lmd * bi
    grd_user_reps = diff * qi + lmd * pu
    grd_item_reps = diff * pu + lmd * qi

    user_bias[user] = user_bias[user] - lr * grd_user_bias
    item_bias[item] = item_bias[item] - lr * grd_item_bias
    user_reps[user] = user_reps[user] - lr * grd_user_reps
    item_reps[item] = item_reps[item] - lr * grd_item_reps


rmse : 3.30011765104686 , epoch : 0 
rmse : 1.171648352711882 , epoch : 1 
rmse : 1.0385912702790645 , epoch : 2 
rmse : 0.9970695943263967 , epoch : 3 
rmse : 0.9716550503620398 , epoch : 4 
rmse : 0.9522125969991326 , epoch : 5 
rmse : 0.9382007904229416 , epoch : 6 
rmse : 0.9265447265742149 , epoch : 7 
rmse : 0.9178697525210575 , epoch : 8 
rmse : 0.9115097496743874 , epoch : 9 


In [ ]:
!pip install surprise

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 772.0/772.0 kB 8.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.3-cp310-cp310-linux_x86_64.whl size=3162745 sha256=d96983543f3bdf8f625a0f343eb92db198ba30255fb3c2abc56f9c241be04637
  Stored in directory: /root/.cache/pip/wheels/a5/ca/a8/4e28def53797fdc4363ca4af740db15a9c2f1595ebc51fb445
Successfully built scikit-surprise


In [ ]:
from surprise import SVD , Reader , Dataset
from surprise.model_selection import train_test_split

reader = Reader(line_format = "user item rating timestamp" , sep = ',' , skip_lines = 1)
data = Dataset.load_from_file("ml-latest-small/ratings.csv", reader = reader)

trainset , testset = train_test_split(data , test_size = 0.1)



In [ ]:
model = SVD(n_factors = 10) # level 차원의 수
model.fit(trainset)
model.predict('24', '3034') # 23 유저가 123 영화

Prediction(uid='24', iid='3034', r_ui=None, est=3.6029392633728, details={'was_impossible': False})

In [ ]:
from surprise import accuracy

train_to_test = trainset.build_testset()
pred_train = model.test(train_to_test)
rmse_train = accuracy.rmse(pred_train)


pred_test = model.test(testset)
rmse_test = accuracy.rmse(pred_test)

print("rmse_train : {} , rmse_test : {}".format(rmse_train , rmse_test))

RMSE: 0.7991
RMSE: 0.8565
rmse_train : 0.79908457724616 , rmse_test : 0.8564526670569853
